# Hands-On Lab: Language Understanding with Recurrent Networks

**Если кто-то до этого не работал с Jupiter Notebooks, то можно почитать вот [здесь](http://jupyter-notebook.readthedocs.io/en/latest/examples/Notebook/What%20is%20the%20Jupyter%20Notebook.html) **

Данный туториал показывает применение рекурентной нейронной сети для обработки текста [Air Travel Information Services](https://catalog.ldc.upenn.edu/LDC95S26) 
(ATIS) задание по слот-тегированию (тегирование отдельных слов в необходимые классы, где классы описанны как "лейблы" в тренировочном наборе данных).

Мы начнем с векторизации данных слов, продолжим их обработкой в рекурентной LSTM и расширим ее для включения соседних слов и параллельного запуска.


Технологии, которые вы будете практиковать:

* описание модели через составление блока слоев(layer block)-удобный способ для конструирования сетей без необходимости описывать математическую формулу     model description by composing layer blocks, a convenient way to compose 
  networks/models without requiring the need to write formulas,
* создание создание блока слоев(layer block)     creating your own layer block
* создание переменных с разной длинной последовательностей (sequence length) в одной нейросети   variables with different sequence lengths in the same network
* тренировка нейросети training the network

Предполагается знание основ глубокого обучения и следующих концептов:

* рекурентные нейросети ([Wikipedia page](https://en.wikipedia.org/wiki/Recurrent_neural_network))
* векторное представление текста ([Wikipedia page](https://en.wikipedia.org/wiki/Word_embedding))

### Prerequisites

Мы предпологаем, что вы уже установили [CNTK](https://www.cntk.ai/pythondocs/setup.html).
Данное руководство требует CNTK V2. Рекомендуется запускать данный туториал на устройстве с GPU(графический процессор), совместимом с технологией [CUDA](http://www.nvidia.ru/object/cuda-parallel-computing-ru.html). Глубокое обучение без GPU это не весело, хех.

#### Скачивание необходимых дата-сетов

В данном руководстве мы будет использовать слегка переработанную версию датасета ATIS. Вы можете скачать его автоматически, запустив код в </i>клетке</i> ниже или вручную по ссылкам ниже.


#### Инструкции для ручного скачивания
Пожалуйста, скачайте блок данных ATIS для [тренировки](https://github.com/Microsoft/CNTK/blob/master/Tutorials/SLUHandsOn/atis.train.ctf) 
и [тест](https://github.com/Microsoft/CNTK/blob/master/Tutorials/SLUHandsOn/atis.test.ctf) 
поместите эти файлы в ту же папку, что и настоящий файл(notebook). Если вы хотите увидеть, как модель предугадывает слова на новых предложениях, которые вы вводите, вам так же понадобятся словари для запросов и слотов(queries and slots).
[queries](https://github.com/Microsoft/CNTK/blob/master/Examples/Text/ATIS/query.wl) 
[slots](https://github.com/Microsoft/CNTK/blob/master/Examples/Text/ATIS/slots.wl) .

In [2]:
import requests

def download(url, filename):
    """ utility to download necessary data """
    response = requests.get(url, stream=True)
    with open(filename, "wb") as handle:
        for data in response.iter_content():
            handle.write(data)

url1 = "https://github.com/Microsoft/CNTK/blob/master/Examples/Tutorials/SLUHandsOn/atis.%s.ctf?raw=true"
url2 = "https://github.com/Microsoft/CNTK/blob/master/Examples/Text/ATIS/%s.wl?raw=true"
urls = [url1%"train", url1%"test", url2%"query", url2%"slots"]

for t in urls:
    filename = t.split('/')[-1].split('?')[0]
    try:
        f = open(filename)
        f.close()
    except IOError:
        download(t, filename)


#### Импорт CNTK и необходимых модулей

CNTK-модуль Python, который содержит некоторые подмодули(`io`, `learner`,`layers`). Мы так же используем NumPy в некоторых случаях, так как результаты, возвращаемые CNTK работают как NumPy-массивы.



In [3]:
import math
import numpy as np
from cntk.blocks import default_options, LSTM, Placeholder, Input        # building blocks
from cntk.layers import Embedding, Recurrence, Dense, BatchNormalization # layers
from cntk.models import Sequential                                       # higher level things
from cntk.utils import ProgressPrinter, log_number_of_parameters
from cntk.io import MinibatchSource, CTFDeserializer
from cntk.io import StreamDef, StreamDefs, INFINITELY_REPEAT, FULL_DATA_SWEEP
from cntk import future_value, combine, Trainer, cross_entropy_with_softmax, classification_error, splice
from cntk.learner import adam_sgd, learning_rate_schedule

## Задание и структура модели

Наша цель - присвоить каждое слово соответствующему слоту(slot tagging). 
Мы используем [ATIS corpus](https://catalog.ldc.upenn.edu/LDC95S26).
ATIS содержит запросы от "человека" к "компьютеру" c домена Air Travel Information Services,
и нашим заданием будет тэгировать каждое слово запроса к соответствующему слоту(единице информации) и определение к какому именно.

Данные, скачанные нами в рабочую папку уже были конвертированы в "CNTK Text Format".
Посмотрим на следующий тестовый файл `atis.test.ctf`:

    19  |S0 178:1 |# BOS      |S1 14:1 |# flight  |S2 128:1 |# O
    19  |S0 770:1 |# show                         |S2 128:1 |# O
    19  |S0 429:1 |# flights                      |S2 128:1 |# O
    19  |S0 444:1 |# from                         |S2 128:1 |# O
    19  |S0 272:1 |# burbank                      |S2 48:1  |# B-fromloc.city_name
    19  |S0 851:1 |# to                           |S2 128:1 |# O
    19  |S0 789:1 |# st.                          |S2 78:1  |# B-toloc.city_name
    19  |S0 564:1 |# louis                        |S2 125:1 |# I-toloc.city_name
    19  |S0 654:1 |# on                           |S2 128:1 |# O
    19  |S0 601:1 |# monday                       |S2 26:1  |# B-depart_date.day_name
    19  |S0 179:1 |# EOS                          |S2 128:1 |# O

Файл имеет 7 столбцов:

* id последовательности (19). Всего 11 вхождений с этим id. Это значит, что эта последовательность состит из 11 слов(tokens);
* столбец `S0`, который содержит индексы слов;
* закомментированный `#` столбец, позволяющий читателю понять, что было введено;
Закомментированные стобцы игнорируются системой. `BOS` и `EOS`-специальные слова, для обозначения начала и конца предложения;
* столбец `S1` - специальный label, который мы будем использовать позже, в последеней части туториала;
* другой закомментированный `#` столбец показывает маркировку(label) читаемую человеком;
* столбец `S2` - маркировка нужного слота в виде индекса;
* еще один столбец-комментарий, поясняющий label.

Задание нейросети состоит в том, чтобы по последовательности(колонка S0) предсказать верную маркировку слота.

Как вы можете видеть, каждому слову на входе присваивается либо пустой label `O`, либо специальный slot-label, начинающийся с   `B-` для первого слова, и с `I-` для каждого дополнительного слова последовательности принадлежащего этому же слоту. 

Мы будем использовать рекурентную модель, состоящую из трёх слоев: embedding(векторизация), рекурентную [LSTM](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) сеть и dense layer для рассчета последующих вероятностей.



    slot label   "O"        "O"        "O"        "O"  "B-fromloc.city_name"
                  ^          ^          ^          ^          ^
                  |          |          |          |          |
              +-------+  +-------+  +-------+  +-------+  +-------+
              | Dense |  | Dense |  | Dense |  | Dense |  | Dense |  ...
              +-------+  +-------+  +-------+  +-------+  +-------+
                  ^          ^          ^          ^          ^
                  |          |          |          |          |
              +------+   +------+   +------+   +------+   +------+   
         0 -->| LSTM |-->| LSTM |-->| LSTM |-->| LSTM |-->| LSTM |-->...
              +------+   +------+   +------+   +------+   +------+   
                  ^          ^          ^          ^          ^
                  |          |          |          |          |
              +-------+  +-------+  +-------+  +-------+  +-------+
              | Embed |  | Embed |  | Embed |  | Embed |  | Embed |  ...
              +-------+  +-------+  +-------+  +-------+  +-------+
                  ^          ^          ^          ^          ^
                  |          |          |          |          |
    w      ------>+--------->+--------->+--------->+--------->+------... 
                 BOS      "show"    "flights"    "from"   "burbank"

Посмотрите на описание сети в языке CNTK и сравните со схемой, представленной выше:
(описания функций [здесь](http://cntk.ai/pythondocs/layerref.html)


In [4]:
# number of words in vocab, slot labels, and intent labels
vocab_size = 943 ; num_labels = 129 ; num_intents = 26    

# model dimensions
input_dim  = vocab_size
label_dim  = num_labels
emb_dim    = 150
hidden_dim = 300

def create_model():
    with default_options(initial_state=0.1):
        return Sequential([
            Embedding(emb_dim),
            Recurrence(LSTM(hidden_dim), go_backwards=False),
            Dense(num_labels)
        ])

Теперь мы готовы к созданию и осмотру модели.



In [5]:
# peek
model = create_model()
print(len(model.layers))
print(model.layers[0].E.shape)
print(model.layers[2].b.value)

3
(-1, 150)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.]


Как вы можете видеть, атрибуты модели полностью доступны через Python. Модель имеет 3 слоя. Первый слой-векторизация/отображение(embedding), доступ к нему осуществляется через параметр `E` (где хранятся embeddings) аналогично любому другому свойству объекта Python. Его свойство .shape содерджит `-1`, что указывает на то, что данный параметр еще не полностью определен. Когда мы решим, какие данные мы будет пропускать через данную сеть, .shape примет значение равное размеру словаря на висходных данных. Мы так же напечатали bias term(что-то вроде погрешности выборки, подробней [здесь](https://www.quora.com/What-does-the-bias-term-represent-in-logistic-regression) и [здесь](http://stackoverflow.com/questions/2480650/role-of-bias-in-neural-networks/2499936#2499936)) для последнего слоя. Bias terms изначально установлены на 0(но мы можем изменить их).  


## CNTK-конфигурация

Для тренировки и тестирования модели в CNTK мы должны создать эту модели и провести уточнения по чтению даты и проведению тренировок и тестов.

Для того, чтобы тренировать нашу сеть мы должны определить:

* как читать данные 
* функции модели, данные на входе и выходе
* специальные параметры для "ученика", такие как статистика обучения

[comment]: <> (For testing ...)

### О данных и об их чтении 

Мы уже посмотрели на данные.
Однако как мы можем сгенерировать их в нужном формате?

Для чтения текста мы будем использовать `CNTKTextFormatReader`. Данный модуль ожидает входные данные в специальном формате, описанном [здесь](https://github.com/Microsoft/CNTK/wiki/CNTKTextFormat-Reader).

Для данного руководства необходимо совершить следующее:
* convert the raw data into a plain text file that contains of TAB-separated столбецs of space-separated text. For example:

  ```
  BOS show flights from burbank to st. louis on monday EOS (TAB) flight (TAB) O O O O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name O B-depart_date.day_name O
  ```

Это должно быть совместимо с выводом команды `paste`.
* конвертируем в специальный текстовый формат CNTK (CTF) с помощью следующей команды:

  ```
  python [CNTK root]/Scripts/txt2ctf.py --map query.wl intent.wl slots.wl --annotated True --input atis.test.txt --output    atis.test.ctf
  ```
  
  
  где три `.wl` файла дают словарь в виде простых текстовых файлов, по одному слову в строке
  
В этих CTF-файлах, наши столбцы уже промаркированы `S0`, `S1`, and `S2`.
Они присоединены к "входам"(inputs) нашей сети через следующие строки кода определяющих "читателя":

In [ ]:
def create_reader(path, is_training):
    return MinibatchSource(CTFDeserializer(path, StreamDefs(
         query         = StreamDef(field='S0', shape=vocab_size,  is_sparse=True),
         intent_unused = StreamDef(field='S1', shape=num_intents, is_sparse=True),  
         slot_labels   = StreamDef(field='S2', shape=num_labels,  is_sparse=True)
     )), randomize=is_training, epoch_size = INFINITELY_REPEAT if is_training else FULL_DATA_SWEEP)

In [ ]:
# peek
reader = create_reader("atis.train.ctf", is_training=True)
reader.streams.keys()

### Тренировка

Мы также должны определить критерий тренировки, в роли которого будет выступать [функция потерь](https://en.wikipedia.org/wiki/Loss_function), а так же метрику ошибок для отслеживания. В коде ниже мы обширно используем `Placeholders`. Важно помнить, что код, который мы писали до этого не выполняет никаких тяжелых вычислений. Он лишь определяет функцию, которую мы хотим использовать на данных во время тренировки и тестирования. Так же, как удобно иметь имена для аргументов при написании обычной функции, удобно иметь `Placeholders`, которые ссылаются на аргументы (или локальные вычисления, которые нужно повторить). В какой то момент, другой код заменить `Placeholders` другими известными величинами, так же как обычная функция вызывается с определенными величинами, привязанными к ее аргументам.


In [6]:
def create_criterion_function(model):
    labels = Placeholder()
    ce   = cross_entropy_with_softmax(model, labels)
    errs = classification_error      (model, labels)
    return combine ([ce, errs]) # (features, labels) -> (loss, metric)

In [8]:
def train(reader, model, max_epochs=16):
    # criterion: (model args, labels) -> (loss, metric)
    #   here  (query, slot_labels) -> (ce, errs)
    criterion = create_criterion_function(model)

    criterion.replace_placeholders({criterion.placeholders[0]: Input(vocab_size),
                                    criterion.placeholders[1]: Input(num_labels)})

    # training config
    epoch_size = 18000        # 18000 samples is half the dataset size 
    minibatch_size = 70
    
    # LR schedule over epochs 
    # In CNTK, an epoch is how often we get out of the minibatch loop to
    # do other stuff (e.g. checkpointing, adjust learning rate, etc.)
    # (we don't run this many epochs, but if we did, these are good values)
    lr_per_sample = [0.003]*4+[0.0015]*24+[0.0003]
    lr_schedule = learning_rate_schedule(lr_per_sample, units=epoch_size)
    
    # Momentum (could also be on a schedule)
    momentum_as_time_constant = 700
    
    # We use a variant of the Adam optimizer which is known to work well on this dataset
    # Feel free to try other optimizers from 
    # https://www.cntk.ai/pythondocs/cntk.learner.html#module-cntk.learner
    learner = adam_sgd(criterion.parameters,
                       lr_per_sample=lr_schedule, momentum_time_constant=momentum_as_time_constant,
                       low_memory=True,
                       gradient_clipping_threshold_per_sample=15, gradient_clipping_with_truncation=True)

    # trainer
    trainer = Trainer(model, criterion.outputs[0], criterion.outputs[1], learner)

    # process minibatches and perform model training
    log_number_of_parameters(model)
    progress_printer = ProgressPrinter(tag='Training')
    #progress_printer = ProgressPrinter(freq=100, first=10, tag='Training') # more detailed logging

    t = 0
    for epoch in range(max_epochs):         # loop over epochs
        epoch_end = (epoch+1) * epoch_size
        while t < epoch_end:                # loop over minibatches on the epoch
            data = reader.next_minibatch(minibatch_size, input_map={  # fetch minibatch
                criterion.arguments[0]: reader.streams.query,
                criterion.arguments[1]: reader.streams.slot_labels
            })
            trainer.train_minibatch(data)                                     # update model with it
            t += data[criterion.arguments[1]].num_samples                     # samples so far
            progress_printer.update_with_trainer(trainer, with_metric=True)   # log progress
        loss, metric, actual_samples = progress_printer.epoch_summary(with_metric=True)

    return loss, metric

### Запуск

Так выглядит готовая модель запуска:

In [ ]:
def do_train():
    global model
    model = create_model()
    reader = create_reader("atis.train.ctf", is_training=True)
    train(reader, model)
do_train()


Код выше показывает, как процесс обучения проходит через данные.
Например, после четрых циклов(epoch) потери достигли 0.22, как было измерено на ~18000 образцах этой выборки, а ошибки возникли в 5% случаев на данных образцах.


Размер выборки-количество образцов, то есть слов, а не предложений, которые были обработаны моделью.

The epoch size is the number of samples--counted as *word tokens*, not sentences--to
process between model checkpoints.

Когда тренировка будет выполнена, вы увидите следующее:
```
Finished Epoch [16]: [Training] loss = 0.058111 * 18014, metric = 1.3% * 18014
```
, где loss (cross entropy)-потери, а metric-ошибка классификации посчитаны в среднем, после последнего "прохода".

На компьютере оснащенном только CPU, тренировка может занять в 4 раза дольше. Вы можете попробовать установить
```
python
emb_dim    = 50 
hidden_dim = 100
```
чтобы уменьшить время обработки, однако модель может не подойти под эти критерии, так же как в тех случаях, когда слои будут больше. 

### Оценка модели

Так же как и функцию train(), мы должны объявить функцию для измерения "аккуратности" нейросети на тестовом сете данных.

In [ ]:
def evaluate(reader, model):
    criterion = create_criterion_function(model)
    criterion.replace_placeholders({criterion.placeholders[0]: Input(num_labels)})

    # process minibatches and perform evaluation
    dummy_learner = adam_sgd(criterion.parameters, 
                             lr_per_sample=1, momentum_time_constant=0, low_memory=True)
    evaluator = Trainer(model, criterion.outputs[0], criterion.outputs[1], dummy_learner)
    progress_printer = ProgressPrinter(tag='Evaluation')

    while True:
        minibatch_size = 1000
        data = reader.next_minibatch(minibatch_size, input_map={  # fetch minibatch
            criterion.arguments[0]: reader.streams.query,
            criterion.arguments[1]: reader.streams.slot_labels
        })
        if not data:                                 # until we hit the end
            break
        metric = evaluator.test_minibatch(data)
        progress_printer.update(0, data[criterion.arguments[1]].num_samples, metric) # log progress
    loss, metric, actual_samples = progress_printer.epoch_summary(with_metric=True)

    return loss, metric

Теперь мы можем измерить аккуратность модели, проходя через все примеры тестового наобра и используя метод ``test_minibatch``, который создан внутри функции ``evaluate``, определенной выше. На данный момент, конструктору Trainer необходим ученик(даже если он используется только для запуска ``test_minibatch``), поэтому необходимо "уточнить" какого-нибудь глупенького ученика:)  


In [ ]:
def do_test():
    reader = create_reader("atis.test.ctf", is_training=False)
    evaluate(reader, model)
do_test()
model.layers[2].b.value

In [ ]:
# load dictionaries
query_wl = [line.rstrip('\n') for line in open('query.wl')]
slots_wl = [line.rstrip('\n') for line in open('slots.wl')]
query_dict = {query_wl[i]:i for i in range(len(query_wl))}
slots_dict = {slots_wl[i]:i for i in range(len(slots_wl))}

# let's run a sequence through
seq = 'BOS flights from new york to seattle EOS'
w = [query_dict[w] for w in seq.split()] # convert to word indices
print(w)
onehot = np.zeros([len(w),len(query_dict)], np.float32)
for t in range(len(w)):
    onehot[t,w[t]] = 1
pred = model.eval({model.arguments[0]:onehot})
print(pred.shape)
best = np.argmax(pred,axis=2)
print(best[0])
list(zip(seq.split(),[slots_wl[s] for s in best[0]]))

## Modifying the Model

In the following, you will be given tasks to practice modifying CNTK configurations.
The solutions are given at the end of this document... but please try without!

### A Word About [`Sequential()`](https://www.cntk.ai/pythondocs/layerref.html#sequential)

Before jumping to the tasks, let's have a look again at the model we just ran.
The model is described in what we call *function-composition style*.
```python
        Sequential([
            Embedding(emb_dim),
            Recurrence(LSTM(hidden_dim), go_backwards=False),
            Dense(num_labels)
        ])
```
You may be familiar with the "sequential" notation from other neural-network toolkits.
If not, [`Sequential()`](https://www.cntk.ai/pythondocs/layerref.html#sequential) is a powerful operation that,
in a nutshell, allows to compactly express a very common situation in neural networks
where an input is processed by propagating it through a progression of layers.
`Sequential()` takes an list of functions as its argument,
and returns a *new* function that invokes these functions in order,
each time passing the output of one to the next.
For example,
```python
	FGH = Sequential ([F,G,H])
    y = FGH (x)
```
means the same as
```
    y = H(G(F(x))) 
```
This is known as ["function composition"](https://en.wikipedia.org/wiki/Function_composition),
and is especially convenient for expressing neural networks, which often have this form:

         +-------+   +-------+   +-------+
    x -->|   F   |-->|   G   |-->|   H   |--> y
         +-------+   +-------+   +-------+

Coming back to our model at hand, the `Sequential` expression simply
says that our model has this form:

         +-----------+   +----------------+   +------------+
    x -->| Embedding |-->| Recurrent LSTM |-->| DenseLayer |--> y
         +-----------+   +----------------+   +------------+

### Task 1: Add Batch Normalization

We now want to add new layers to the model, specifically batch normalization.

Batch normalization is a popular technique for speeding up convergence.
It is often used for image-processing setups, for example our other [hands-on lab on image
recognition](./Hands-On-Labs-Image-Recognition).
But could it work for recurrent models, too?
  
So your task will be to insert batch-normalization layers before and after the recurrent LSTM layer.
If you have completed the [hands-on labs on image processing](https://github.com/Microsoft/CNTK/blob/master/bindings/python/tutorials/CNTK_201B_CIFAR-10_ImageHandsOn.ipynb),
you may remember that the [batch-normalization layer](https://www.cntk.ai/pythondocs/layerref.html#batchnormalization-layernormalization-stabilizer) has this form:
```
    BatchNormalization()
```
So please go ahead and modify the configuration and see what happens.

If everything went right, you will notice improved convergence speed (`loss` and `metric`)
compared to the previous configuration.

In [ ]:
# Your task: Add batch normalization
def create_model():
    with default_options(initial_state=0.1):
        return Sequential([
            Embedding(emb_dim),
            Recurrence(LSTM(hidden_dim), go_backwards=False),
            Dense(num_labels)
        ])

do_train()
do_test()

### Task 2: Add a Lookahead 

Our recurrent model suffers from a structural deficit:
Since the recurrence runs from left to right, the decision for a slot label
has no information about upcoming words. The model is a bit lopsided.
Your task will be to modify the model such that
the input to the recurrence consists not only of the current word, but also of the next one
(lookahead).

Your solution should be in function-composition style.
Hence, you will need to write a Python function that does the following:

* takes no input arguments
* creates a placeholder (sequence) variable
* computes the "next value" in this sequence using the `future_value()` operation and
* concatenates the current and the next value into a vector of twice the embedding dimension using `splice()`

and then insert this function into `Sequential()`'s list right after the embedding layer.

In [ ]:
# Your task: Add lookahead
def create_model():
    with default_options(initial_state=0.1):
        return Sequential([
            Embedding(emb_dim),
            Recurrence(LSTM(hidden_dim), go_backwards=False),
            Dense(num_labels)
        ])
do_train()
do_test()

### Task 3: Bidirectional Recurrent Model

Aha, knowledge of future words help. So instead of a one-word lookahead,
why not look ahead until all the way to the end of the sentence, through a backward recurrence?
Let us create a bidirectional model!

Your task is to implement a new layer that
performs both a forward and a backward recursion over the data, and
concatenates the output vectors.

Note, however, that this differs from the previous task in that
the bidirectional layer contains learnable model parameters.
In function-composition style,
the pattern to implement a layer with model parameters is to write a *factory function*
that creates a *function object*.

A function object, also known as [*functor*](https://en.wikipedia.org/wiki/Function_object), is an object that is both a function and an object.
Which means nothing else that it contains data yet still can be invoked as if it was a function.

For example, `Dense(outDim)` is a factory function that returns a function object that contains
a weight matrix `W`, a bias `b`, and another function to compute 
`input @ W + b.` (This is using 
[Python 3.5 notation for matrix multiplication](https://docs.python.org/3/whatsnew/3.5.html#whatsnew-pep-465).
In Numpy syntax it is `input.dot(W) + b`).
E.g. saying `Dense(1024)` will create this function object, which can then be used
like any other function, also immediately: `Dense(1024)(x)`. 

Let's look at an example for further clarity: Let us implement a new layer that combines
a linear layer with a subsequent batch normalization. 
To allow function composition, the layer needs to be realized as a factory function,
which could look like this:

```python
def DenseLayerWithBN(dim):
    F = Dense(dim)
    G = BatchNormalization()
    x = Placeholder()
    apply_x = G(F(x))
    return apply_x
```

Invoking this factory function will create `F`, `G`, `x`, and `apply_x`. In this example, `F` and `G` are function objects themselves, and `apply_x` is the function to be applied to the data.
Thus, e.g. calling `DenseLayerWithBN(1024)` will
create an object containing a linear-layer function object called `F`, a batch-normalization function object `G`,
and `apply_x` which is the function that implements the actual operation of this layer
using `F` and `G`. It will then return `apply_x`. To the outside, `apply_x` looks and behaves
like a function. Under the hood, however, `apply_x` retains access to its specific instances of `F` and `G`.

Now back to our task at hand. You will now need to create a factory function,
very much like the example above.
You shall create a factory function
that creates two recurrent layer instances (one forward, one backward), and then defines an `apply_x` function
which applies both layer instances to the same `x` and concatenate the two results.

Allright, give it a try! To know how to realize a backward recursion in CNTK,
please take a hint from how the forward recursion is done.
Please also do the following:
* remove the one-word lookahead you added in the previous task, which we aim to replace; and
* make sure each LSTM is using `hidden_dim//2` outputs to keep the total number of model parameters limited.

In [ ]:
# Your task: Add bidirectional recurrence
def create_model():
    with default_options(initial_state=0.1):  
        return Sequential([
            Embedding(emb_dim),
            Recurrence(LSTM(hidden_dim), go_backwards=False),
            Dense(num_labels)
        ])
do_train()
do_test()

Works like a charm! This model achieves 2.1%, a tiny bit better than the lookahead model above.
The bidirectional model has 40% less parameters than the lookahead one. However, if you go back and look closely
you may find that the lookahead one trained about 30% faster.
This is because the lookahead model has both less horizontal dependencies (one instead of two
recurrences) and larger matrix products, and can thus achieve higher parallelism.

### Solution 1: Adding Batch Normalization

In [ ]:
def create_model():
    with default_options(initial_state=0.1):
        return Sequential([
            Embedding(emb_dim),
            BatchNormalization(),
            Recurrence(LSTM(hidden_dim), go_backwards=False),
            BatchNormalization(),
            Dense(num_labels)
        ])

do_train()
do_test()

### Solution 2: Add a Lookahead

In [ ]:
def OneWordLookahead():
    x = Placeholder()
    apply_x = splice ([x, future_value(x)])
    return apply_x

def create_model():
    with default_options(initial_state=0.1):
        return Sequential([
            Embedding(emb_dim),
            OneWordLookahead(),
            Recurrence(LSTM(hidden_dim), go_backwards=False),
            Dense(num_labels)        
        ])

do_train()
do_test()

### Solution 3: Bidirectional Recurrent Model

In [ ]:
def BiRecurrence(fwd, bwd):
    F = Recurrence(fwd)
    G = Recurrence(bwd, go_backwards=True)
    x = Placeholder()
    apply_x = splice ([F(x), G(x)])
    return apply_x 

def create_model():
    with default_options(initial_state=0.1):
        return Sequential([
            Embedding(emb_dim),
            BiRecurrence(LSTM(hidden_dim//2), LSTM(hidden_dim//2)),
            Dense(num_labels)
        ])

do_train()
do_test()